# Descargar automáticamente del SMN
A partir de la ejecución del siguiente código: 
1. Se descargan los archivos de datos horarios del Servicio Meteorológico Nacional de las fechas seleccionadas.
2. Se estandarizan los mismos y compila en una única tabla.
3. Se los exporta como archivo CSV.

Fuente: https://www.smn.gob.ar/descarga-de-datos

### DH. Datos Horarios
Dato meteorológico de todas las estaciones de superficie del país
Datos de temperatura, presión, viento y humedad de las estaciones meteorológicas de todo el país.

In [1]:
#Cargar librerías
import requests
import pandas as pd
from datetime import datetime, date, timedelta
from os import  mkdir, rmdir, remove

In [2]:
# Descarga de datos
dir = "meteodata" #carpeta de destino de los archivos
mkdir(str(dir)) 

sdate = date(2018, 1, 1)   # fecha de inicio (año, mes, día)
edate = date(2018, 1, 7)   # fecha de fin (año, mes, día)

delta = edate - sdate

for i in range(delta.days + 1):
    date = str(sdate + timedelta(days=i)).replace('-', '')
    url = 'https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=observaciones/datohorario' + date + '.txt'
    r = requests.get(url, allow_redirects=True)
    if 'El archivo no existe.' in str(r.content):
        print(date + " does not exist")
    else:
        with open('./' + dir + '/' + date + '.txt', 'wb') as f: f.write(r.content) 

In [3]:
#Filtrado y combinación de archivos
estacion = 'ROSARIO AERO' #filtro de selección de estación

out = pd.DataFrame(columns = ['Fecha', 'Hora', 'Temp[ºC]', 'Hum[%]', 'Pres[hPa]', 'VDir[gr]', 'VVel[km/h]', 'Estacion'])

for i in range(delta.days + 1):
    date = str(sdate + timedelta(days=i)).replace('-', '')
    df = pd.read_fwf('./' + dir + '/'  + date + '.txt', header = None, index_col = False, 
                names = ['Fecha', 'Hora', 'Temp[ºC]', 'Hum[%]', 'Pres[hPa]', 'VDir[gr]', 'VVel[km/h]', 'Estacion'])
    df1 = df[df['Estacion'] == estacion] 
    out = out.append(df1, ignore_index=True)
    
display(out.head())
display(out.tail())

,Fecha,Hora,Temp[ºC],Hum[%],Pres[hPa],VDir[gr],VVel[km/h],Estacion
0,01012018,0,20.3,77,1011.5,110,20,ROSARIO AERO
1,01012018,1,20.0,78,1011.5,140,9,ROSARIO AERO
2,01012018,2,18.8,91,1011.4,230,9,ROSARIO AERO
3,01012018,3,18.0,93,1011.2,180,7,ROSARIO AERO
4,01012018,4,17.5,95,1010.6,180,9,ROSARIO AERO


,Fecha,Hora,Temp[ºC],Hum[%],Pres[hPa],VDir[gr],VVel[km/h],Estacion
162,07012018,19,30.5,32,1015.4,50,9,ROSARIO AERO
163,07012018,20,28.6,40,1015.5,50,6,ROSARIO AERO
164,07012018,21,26.5,50,1015.8,70,7,ROSARIO AERO
165,07012018,22,24.5,61,1016.3,70,6,ROSARIO AERO
166,07012018,23,23.6,67,1016.8,70,6,ROSARIO AERO


In [4]:
#Estandarización
df = out

df['Horario'] = df['Fecha'].map(str) + ' ' + df['Hora'].map(str) + ':00'
df['Horario'] = pd.to_datetime(df['Horario'], format ='%d%m%Y %H:%M') 

df['Temp[ºC]'] = df['Temp[ºC]'].astype(float)
df['Pres[hPa]'] = df['Pres[hPa]'].astype(float)
df['Hum[%]'] = df['Hum[%]'].astype(int)
df['VDir[gr]'] = df['VDir[gr]'].astype(int)
df['VVel[km/h]'] = df['VVel[km/h]'].astype(int)

df = df[['Horario', 'Temp[ºC]', 'Hum[%]', 'Pres[hPa]', 'VDir[gr]', 'VVel[km/h]']]

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Horario     167 non-null    datetime64[ns]
 1   Temp[ºC]    167 non-null    float64       
 2   Hum[%]      167 non-null    int32         
 3   Pres[hPa]   167 non-null    float64       
 4   VDir[gr]    167 non-null    int32         
 5   VVel[km/h]  167 non-null    int32         
dtypes: datetime64[ns](1), float64(2), int32(3)
memory usage: 6.0 KB
None


In [5]:
#Borra archivos anteriores y exporta la tabla
file = 'datosRosario.csv' #nombre del archivo de destino

df.to_csv(file, index = False) 

for i in range(delta.days + 1):
    date = str(sdate + timedelta(days=i)).replace('-', '')
    remove('./' + dir + '/' + date + '.txt')
rmdir(str(dir))